In [373]:
# Import required libraries

from psaw import PushshiftAPI
import praw
import datetime as dt
import pandas as pd
import csv
from pathlib import Path


In [374]:
  # Initiate api
  
  api = PushshiftAPI()

In [375]:
# use serach_submissions to pull only posts in wallstreeetbets that contain CEI in it

subs = api.search_submissions(
    q='CEI',
    subreddit='wallstreetbets'
)

In [376]:
# Create dataframe to hold results

subs_df = pd.DataFrame([submission.d_ for submission in subs])

C:\Users\medin\Anaconda3\envs\dev\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [377]:
# Determining size of data
subs_df.shape

(380, 84)

In [378]:
# View all columns to choose required columns
subs_df.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_is_blocked',
       'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post',
       'contest_mode', 'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_created_from_ads_ui', 'is_crosspostable', 'is_meta',
       'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable',
       'is_self', 'is_video', 'link_flair_background_color',
       'link_flair_css_class', 'link_flair_richtext', 'link_flair_template_id',
       'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked',
       'media_only', 'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'post_hint',
       'preview', 'pwls', 'removed_by_category', 'retrieved_on', 'score',
       'selftext', 'send_replies', 'spoiler', 'stickie

In [379]:
# Testing column names to review data 
subs_df[['author','title','selftext','score','created_utc']].sample(1)

,author,title,selftext,score,created_utc
178,timothymoss84,Future of $CEI,Camber Energy secured a patent last month and...,1,1631720444


In [380]:
# Convert datetime to readable format
subs_df['date'] = pd.to_datetime(subs_df['created_utc'], utc =True, unit='s')

In [381]:
# Create new dataframe to hold required columns
subs_df_2 = subs_df[['title','selftext','date','score','num_comments']]

display(subs_df_2.head(5))
display(subs_df_2.tail(5))


In [383]:
# Convert format to required format
subs_df_2['date']= subs_df_2.loc[:,'date'].apply(lambda x:pd.Timestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

C:\Users\medin\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [391]:

# Remove emojis

subs_df_2 = subs_df_2.astype(str).apply(lambda x: x.str.encode('ascii','ignore').str.decode('ascii'))

# Remove URLs

subs_df_2 = subs_df_2.replace(to_replace=r'https?:\/\/.*[\r\n]*',value='',regex=True)
# Remove [removed]

subs_df_2 = subs_df_2.replace('[removed]','')

# Removed posts without a date

subs_df_2 = subs_df_2.dropna(subset=['date'])


In [392]:
# Export dataframe into a csv
subs_df_2.to_csv(Path('./Resources/reddit_data.csv'),encoding='utf-8', index=None, header=True)
subs_df_2.tail(10)